<a href="https://colab.research.google.com/github/bingxiaochen/ST-554-Project1/blob/main/Task1/ST_554_P1_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Bing Chen

Purpose: This programs is written for task 1 of project 1.

Date: 2-10-2026

# Read in data

To use the air quality data set available at the UCI machine learning repository, we want to first install the `ucimlrepo` library.

In [47]:
!pip install ucimlrepo

After installing it, import it in to use it. And, go ahead to import all other modules we might use.

In [48]:
import ucimlrepo as uci
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

Now, we are ready to read in the air quality data from the library.

In [49]:
air_quality = uci.fetch_ucirepo(id=360)
air_quality.data.features

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


Now, let's do some data cleaning.

Let's check the type of each column.



In [50]:
air_quality.data.features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   float64
 3   PT08.S1(CO)    9357 non-null   int64  
 4   NMHC(GT)       9357 non-null   int64  
 5   C6H6(GT)       9357 non-null   float64
 6   PT08.S2(NMHC)  9357 non-null   int64  
 7   NOx(GT)        9357 non-null   int64  
 8   PT08.S3(NOx)   9357 non-null   int64  
 9   NO2(GT)        9357 non-null   int64  
 10  PT08.S4(NO2)   9357 non-null   int64  
 11  PT08.S5(O3)    9357 non-null   int64  
 12  T              9357 non-null   float64
 13  RH             9357 non-null   float64
 14  AH             9357 non-null   float64
dtypes: float64(5), int64(8), object(2)
memory usage: 1.1+ MB


Now, we want to remove any observations where the `C6H6(GT)` or `CO(GT)` are -200 as these represent missing values. We will save the result as a new dataset. We can compare the shape of these datasets to see how many observations were removed.

In [51]:
air_clean = air_quality.data.features.loc[
    (air_quality.data.features["C6H6(GT)"] != -200) |
    (air_quality.data.features["CO(GT)"] != -200)
]

print(air_quality.data.features.shape)
print(air_clean.shape)


(9357, 15)
(9321, 15)


Looks like 36 observations are removed.

Now, we are ready to make predictions of C6H6(GT).

# Prediction of C6H6(GT)

We want to be able to predict a value of `C6H6(GT)` for a new sample. We'll quantify the quality of the prediction through a loss function.


So, first, let's understand what a loss function is.

Suppose there is some prediction (call it $c$). The squared error loss for a data point (call it $y_1$) and $c$ is $L(y_1, c) = (y_1 − c)^2$

For a given set of data $(y_1,..., y_n)$, we could consider the mean squared error:

Root Mean Square Error = $RMSE(c) = \sqrt{\frac{1}{n}\sum^n_{i=1}(y_1 − c)^2}$


As a prediction, we now want to choose the value of $c$ that minimizes this equation.



We'll use numerical methods to find the optimal value:

1. Grid search
2. Gradient descent

We'll implement these two algorithms for two equations representing $c$:

- we don’t consider any data other than  𝑦 , and c is just a constant that minimizes the function.
- we consider a linear equation (SLR model) of one other numeric variable.


## Grid Search

Grid search method follows the steps:  

1. Create a grid of values for $c$.
2. Find the RMSE for each value of $c$.
3. Determine which value of $c$ gives the optimal (smallest) RMSE.
4. And finally, report that as the prediction!


Now let's implement it together.


To use a reasonable grid for $c$, let's consider using the first and third quartiles for the `C6H6(GT)`, since $c$ should live where most of the data are.


In [52]:
q1 = air_clean["C6H6(GT)"].quantile(0.25)
q3 = air_clean["C6H6(GT)"].quantile(0.75)

q1, q3

(np.float64(4.0), np.float64(13.7))

Now, create a grid of $c$ values using the quartiles. Let's create 1000 evenly spaced candidate values for $c$.

In [53]:
c_grid = np.linspace(q1, q3, 1000)

Next step is to find the RMSE for each value of c. Let's create a function to do that.

In [54]:
def find_rmse(y, c_grid):
    """
    Find the RMSE for each value of c.

    Parameters
    ----------
    y : array-like
        The response variable.
    c_grid : array-like
        The grid of c values.
    """

    y = np.asarray(y)
    c_grid = np.asarray(c_grid)

    rmse_values = np.sqrt(np.mean((y[:, None] - c_grid) ** 2, axis=0))

    return rmse_values

Let's test out the function before moving to the next step. Pass the $c$ grid we created into the function.

In [55]:
rmse = find_rmse(air_clean["C6H6(GT)"], c_grid)
rmse

array([39.52964502, 39.52997896, 39.53031528, 39.53065398, 39.53099507,
       39.53133853, 39.53168438, 39.53203261, 39.53238322, 39.53273622,
       39.53309159, 39.53344935, 39.53380949, 39.53417201, 39.53453691,
       39.53490419, 39.53527385, 39.5356459 , 39.53602032, 39.53639713,
       39.53677632, 39.53715789, 39.53754184, 39.53792817, 39.53831688,
       39.53870797, 39.53910144, 39.5394973 , 39.53989553, 39.54029614,
       39.54069914, 39.54110451, 39.54151227, 39.5419224 , 39.54233492,
       39.54274981, 39.54316708, 39.54358674, 39.54400877, 39.54443319,
       39.54485998, 39.54528915, 39.5457207 , 39.54615463, 39.54659094,
       39.54702963, 39.5474707 , 39.54791415, 39.54835997, 39.54880818,
       39.54925876, 39.54971173, 39.55016707, 39.55062479, 39.55108488,
       39.55154736, 39.55201221, 39.55247945, 39.55294906, 39.55342105,
       39.55389541, 39.55437216, 39.55485128, 39.55533278, 39.55581666,
       39.55630291, 39.55679155, 39.55728256, 39.55777594, 39.55

Good, looks like it's working correctly. Now, use a for loop to find which value of $c$ gives the smallest RMSE. We initialize with the first RMSE and its $c$, loop through the rest, every time we find a smaller RMSE, we update both values. Note that `rmse[i] corresponds to c_grid[i]`.

In [56]:
smallest_rmse = rmse[0]
best_c = c_grid[0]

for i in range(1, len(rmse)):
    if rmse[i] < smallest_rmse:
        smallest_rmse = rmse[i]
        best_c = c_grid[i]

smallest_rmse, best_c

(np.float64(39.52964502158688), np.float64(4.0))

There! $c = 4$ is our prediction!

Now, let's wrap all of that in a function.

In [57]:
def grid_search(y, c_grid):

    """
    Find the the optimal constant prediction for your response variable.

    Parameters
    ----------
    y : array-like
        The response variable.
    c_grid : array-like
        The grid of c values.
    """

    rmse = find_rmse(y, c_grid)

    smallest_rmse = rmse[0]
    best_c = c_grid[0]

    for i in range(1, len(rmse)):
        if rmse[i] < smallest_rmse:
            smallest_rmse = rmse[i]
            best_c = c_grid[i]

    return best_c, print(str(best_c) + " is your optimal constant prediction!")



Test our function with `C6H6(GT)` and `PT08.S1(CO)` variable.  

In [58]:
grid_search(air_clean["C6H6(GT)"], c_grid)

4.0 is your optimal constant prediction!


(np.float64(4.0), None)

The calculus based answer for this should comes out to be the sample mean. Let's check!

In [63]:
np.mean(air_clean["C6H6(GT)"])

np.float64(2.6453384829953874)

Note that they don't align because the sample mean falls outside of the interquatile range (which suggest that the data is left-skewed). Let's update the grid to be wider and try it again.

In [64]:
c_grid_c6h6_2 = np.linspace(air_clean["C6H6(GT)"].min(),
    air_clean["C6H6(GT)"].max(),
    1000)

In [65]:
grid_search(air_clean["C6H6(GT)"], c_grid_c6h6_2)

2.7243243243243 is your optimal constant prediction!


(np.float64(2.7243243243243), None)

Now, it's very close!

Update our $c$ grid with values between Q1 and Q3 of `PT08.S1(CO)` variable in order to make a prediction on it.

In [59]:
c_grid_pt08 = np.linspace(air_clean["PT08.S1(CO)"].quantile(0.25),
                          air_clean["PT08.S1(CO)"].quantile(0.75), 1000)

In [60]:
grid_search(air_clean["PT08.S1(CO)"], c_grid_pt08)

1053.7937937937938 is your optimal constant prediction!


(np.float64(1053.7937937937938), None)

In [61]:
np.mean(air_clean["C6H6(GT)"])

np.float64(2.6453384829953874)

Great! Now let's implement grid search method using y and another numeric variable x. We'll use grid search to find the optimal
pair of values for $b_0$ (intercept) and $b_1$ (slope) using `PT08.S1(CO)` as our $x$ variable and `C6H6(GT)` as our $y$ variable. For each observation ($i$), our prediction is given by $c_i = b_0 + b_1x_i$. So, note that our grid now has two-dimensions.

This time, consider a grid of $b_0$ and $b_1$ values as follows:
- Use $b_0$ values from -25 to -15 with increments of 0.1.
- Use $b_1$ values from -5 to 5 with increments of 0.01


Let's create a function that takes in an x and and y column and outputs the optimal values (optimal $b_0$ and $b_1$ combination).

The RMSE formula is a bit different this time since $c_i = b_0 + b_1x_i$. So, the updated RMSE formula is: $RMSE(c) = \sqrt{\frac{1}{n}\sum^n_{i=1}(y_i − (b_0 + b_1x_i)^2}$


Then we will use these values to predict a new `C6H6(GT)` for a `PT08.S1(CO)` of 946, 1075, and 1246.

In [ ]:
def grid_search_SLR(y, x):

    """
    Find the the optimal intercept and slope for your linear equation of
    the response and explanatory variable.

    Parameters
    ----------
    y : array-like
        The response variable.
    x : array-like
        The explanatory variable.
    """

    y = np.asarray(y)
    x = np.asarray(x)

    b0_grid = np.arange(-25, -15, 0.1)
    b1_grid = np.arange(-5, 5, 0.01)

    rmse = find_rmse(y, c_grid)

    smallest_rmse = rmse[0]
    best_c = c_grid[0]

    for i in range(1, len(rmse)):
        if rmse[i] < smallest_rmse:
            smallest_rmse = rmse[i]
            best_c = c_grid[i]

    return best_c, print(str(best_c) + " is your optimal constant prediction!")